In [1]:
# Useful starting lines
%matplotlib inline

import random
from datetime import datetime

import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

# Support Vector Machines
## Classification Using SVM
Load dataset. We will re-use the CERN dataset from project 1, available from https://inclass.kaggle.com/c/epfml-project-1/data

In [2]:
from helpers import load_csv_data

DATA_TRAIN_PATH = 'data/train.csv'

y, X, ids = load_csv_data(DATA_TRAIN_PATH, sub_sample=True)
print(y.shape, X.shape)

(5000,) (5000, 30)


## Prepare cost and prediction functions

In [3]:
def hinge_loss(y, X, w):
    return max(0, 1 - y.T.dot(X.dot(w)))

In [36]:
def calculate_primal_objective(y, X, w, lambda_):
    """compute the full cost (the primal objective), that is loss plus regularizer.
    X: the full dataset matrix, shape = (num_examples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_examples)
    w: shape = (num_features)
    """
    return hinge_loss(y, X, w) + lambda_ / 2 * w.T.dot(w)

In [37]:
def calculate_accuracy(y, X, w):
    """compute the training accuracy on the training set (can be called for test set as well).
    X: the full dataset matrix, shape = (num_examples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_examples)
    w: shape = (num_features)
    """
    return np.sum(y == 2*(X.dot(w) > 0) - 1) / y.shape[0]

## Stochastic Gradient Descent for SVM

Compute the (stochastic) subgradient for the n-th summand of the SVM optimization objective

In [38]:
def calculate_stochastic_gradient(y, X, w, lambda_, n, num_examples):
    """compute the stochastic gradient of loss plus regularizer.
    X: the dataset matrix, shape = (num_examples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_examples)
    w: shape = (num_features)
    n: the index of the (one) datapoint we have sampled
    num_examples: N
    """
    # ***************************************************
    # INSERT YOUR CODE HERE
    # TODO
    # ***************************************************
    # Be careful about the constant N (size) term!
    # The complete objective for SVM is a sum, not an average as in earlier SGD examples!
    x_n, y_n = X[n], y[n]
    alpha_n = float(y_n * x_n @ w < 1)    
    return num_examples * np.squeeze(- alpha_n * y_n * x_n.T) + lambda_ * w

Implement stochastic gradient descent: Pick a data point uniformly at random and update w based on the gradient for the n-th summand of the objective

In [39]:
def sgd_for_svm_demo(y, X):
    
    max_iter = 100000
    gamma = 1
    lambda_ = 0.01
    
    num_examples, num_features = X.shape
    w = np.zeros(num_features)
    
    for it in range(max_iter):
        # n = sample one data point uniformly at random data from x
        n = random.randint(0,num_examples-1)
        
        grad = calculate_stochastic_gradient(y, X, w, lambda_, n, num_examples)
        w -= gamma/(it+1) * grad
        
        if it % 10000 == 0:
            cost = calculate_primal_objective(y, X, w, lambda_)
            print("iteration={i}, cost={c}".format(i=it, c=cost))
    
    print("training accuracy = {l}".format(l=calculate_accuracy(y, X, w)))

sgd_for_svm_demo(y, X)

iteration=0, cost=2905653204391.2656
iteration=10000, cost=14394132233.829668
iteration=20000, cost=12816854773.726545
iteration=30000, cost=11933748579.61534
iteration=40000, cost=11332877788.348251
iteration=50000, cost=10934748116.066883
iteration=60000, cost=10570378634.602715
iteration=70000, cost=10272473099.824507
iteration=80000, cost=10043857968.64704
iteration=90000, cost=9857773914.784016
training accuracy = 0.696


## Coordinate Descent (Ascent) for SVM

Compute the closed-form update for the n-th variable alpha, in the dual optimization problem, given alpha and the current corresponding w

In [40]:
def calculate_coordinate_update(y, X, lambda_, alpha, w, n):
    """compute a coordinate update (closed form) for coordinate n.
    X: the dataset matrix, shape = (num_examples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_examples)
    w: shape = (num_features)
    n: the coordinate to be updated
    """
    x_n, y_n = X[n], y[n]
    old_alpha_n = np.copy(alpha[n])
    
    g = (y_n * x_n.dot(w) - 1)

    if old_alpha_n == 0:
        g = min(g, 0)
    elif old_alpha_n == 1.0:
        g = max(g, 0)
    else:
        g = g
    if g != 0:
        alpha[n] = min(
            max(old_alpha_n - lambda_ * g / (x_n.T.dot(x_n)), 0.0),
            1.0)
    
        # compute the corresponding update on the primal vector w
        w += 1.0 / lambda_ * (alpha[n] - old_alpha_n) * y_n * x_n
    return w, alpha

In [41]:
def calculate_dual_objective(y, X, w, alpha, lambda_):
    """calculate the objective for the dual problem."""
    return np.sum(alpha) - lambda_ / 2.0 * w.T.dot(w)

In [42]:
def coordinate_descent_for_svm_demo(y, X):
    max_iter = 100000
    lambda_ = 0.01

    num_examples, num_features = X.shape
    w = np.zeros(num_features)
    alpha = np.zeros(num_examples)
    
    for it in range(max_iter):
        # n = sample one data point uniformly at random data from x
        n = random.randint(0,num_examples-1)
        
        w, alpha = calculate_coordinate_update(y, X, lambda_, alpha, w, n)
            
        if it % 10000 == 0:
            # primal objective
            primal_value = calculate_primal_objective(y, X, w, lambda_)
            # dual objective
            dual_value = calculate_dual_objective(y, X, w, alpha, lambda_)
            # primal dual gap
            duality_gap = primal_value - dual_value
            print('iteration=%i, primal:%.5f, dual:%.5f, gap:%.5f'%(
                    it, primal_value, dual_value, duality_gap))
    print("training accuracy = {l}".format(l=calculate_accuracy(y, X, w)))

coordinate_descent_for_svm_demo(y, X)

iteration=0, primal:0.00000, dual:0.00000, gap:0.00000
iteration=10000, primal:0.00000, dual:0.00010, gap:-0.00010
iteration=20000, primal:0.00000, dual:0.00019, gap:-0.00019
iteration=30000, primal:0.00000, dual:0.00027, gap:-0.00027
iteration=40000, primal:0.00000, dual:0.00036, gap:-0.00036
iteration=50000, primal:0.00000, dual:0.00045, gap:-0.00044
iteration=60000, primal:284.81998, dual:0.00053, gap:284.81945
iteration=70000, primal:0.00000, dual:0.00062, gap:-0.00061
iteration=80000, primal:1082.64662, dual:0.00070, gap:1082.64592
iteration=90000, primal:0.00000, dual:0.00079, gap:-0.00079
training accuracy = 0.6926
